<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/master/notebooks/RtGender-Notebooks/RtGender_Annotations_Sentiment_PET_prep_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RtGender - Annotations - Sentiment w/ PET


In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = r'/content/drive/MyDrive/w266'
except ModuleNotFoundError:
  path = r'data'

Mounted at /content/drive


<a id='section01'></a>
## Load Modules

In [2]:
%%capture
!pip install -q transformers
!pip install datasets
import datasets 
from datasets import load_dataset, Dataset, DatasetDict

In [3]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
import logging
logging.basicConfig(level=logging.ERROR)
import itertools

In [4]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
# import eda script from github
!git clone https://github.com/timoschick/pet

# import fucntions from scalar_adjs
import sys

# sys.path is a list of absolute path strings
sys.path.append('/content/pet/')


Cloning into 'pet'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 173 (delta 60), reused 54 (delta 48), pack-reused 85
Receiving objects: 100% (173/173), 126.22 KiB | 9.71 MiB/s, done.
Resolving deltas: 100% (98/98), done.


<a id='section02'></a>
# Import and Reshape Data

In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_dev.csv')

print('train_shape: ',train_df.shape)
print('dev_shape: ',dev_df.shape)
train_df

train_shape:  (10746, 9)
dev_shape:  (2303, 9)


,Unnamed: 0,source,op_gender,post_text,response_text,sentiment,relevance,label,labels_4
0,3845,facebook_congress,W,Im reading the 3/1 GAO report that finds billi...,Thank you Congresswoman Bass. Keep up the grea...,Positive,ContentPoster,2,2
1,9743,fitocracy,M,Being followed by the famous DBJ? Quite an honor.,"Well, I am very honored you feel so honored",Positive,Content,2,2
2,13041,ted,W,"Penelope Boston gave a talk about Planets, exp...",Her opinions seem driven by wishful thinking. ...,Mixed,Content,1,3
3,4265,facebook_congress,W,Congress must act to help the 41 million Ameri...,There's no other way out of the enormity excep...,Positive,Content,2,2
4,13145,ted,W,"Pardis Sabeti gave a talk about Africa, big pr...",What were the benefits of the larger community...,Mixed,Content,1,3
...,...,...,...,...,...,...,...,...,...
10741,14128,ted,M,"Bjarke Ingels gave a talk about architecture, ...",Brillant!! Ingels has a terrific future ahead ...,Positive,Content,2,2
10742,5589,facebook_congress,W,I was honored to meet with Eliseo Medina and F...,The Democrats view this as another way to use ...,Negative,Content,0,0
10743,10672,reddit,W,SO YOU LIKE STACKING CUPS?! DO WE HAVE A GREAT...,Is this real?? Well at least this kid will be ...,Mixed,Content,1,3
10744,4839,facebook_congress,M,Try this Brian Schatz FB bumper sticker - an e...,EH BRIAN WEA MY STICKA N WAT OBAMA STAY ON UM ...,Neutral,Irrelevant,1,1


In [7]:
# there are NaNs in the dev dataset remove 
nan_values = dev_df[dev_df.isna().any(axis=1)] 
print(nan_values)

# return without missing values in response_text
dev_df.dropna(subset = ["response_text"], inplace=True)

print("Train shape", train_df.shape)
print("Dev shape", dev_df.shape)

      Unnamed: 0         source op_gender  ...   relevance label labels_4
830         2576  facebook_wiki         M  ...  Irrelevant     1        1
1664        2722  facebook_wiki         W  ...  Irrelevant     1        1

[2 rows x 9 columns]
Train shape (10746, 9)
Dev shape (2301, 9)


In [8]:
def training_size(train_data, x):
    '''Selects x amount of examples from the training data without replacement
    The function samples the same amount of labels from the classes.
    Parameters:
    train_data: training dataframe
    x: number of training examples to sample
    We expect that x <= 1,000, so we can sample without replacement all classes. 
    '''  
    df0 = train_data[train_data['labels_4']==0].sample(x//4, replace=False, random_state=12345)
    df1 = train_data[train_data['labels_4']==1].sample(x//4, replace=False, random_state=12345)
    df2 = train_data[train_data['labels_4']==2].sample(x//4, replace=False, random_state=12345)
    df3 = train_data[train_data['labels_4']==3].sample(x//4, replace=False, random_state=12345)
    
    # create the training dataframe
    train = pd.concat([df0,df1,df2,df3], axis=0)[['response_text', 'labels_4']]
    
    # index of the elements included in the training dataframe
    index_train = train.index.to_list()
    
    # the unlabeled datafrane are all training data not used in the train dataframe
    unlabeled = train_data.drop(index=index_train)[['response_text', 'labels_4']]
     
    # shuffle rows
    train = train.sample(frac=1,  random_state=12345)
    unlabeled = unlabeled.sample(frac=1, random_state=12345)
    
    return train, unlabeled

In [9]:
# create training, unlabeled dataframes
labeled_examples=12
data = training_size(train_df, labeled_examples)
train = data[0]
unlabeled = data[1]

In [10]:
# convert labels to strings
train['labels_4'] = train['labels_4'].astype('str')
dev_df['labels_4'] = dev_df['labels_4'].astype('str')

In [11]:
# save data

path = '/content/drive/MyDrive/w266'
train_name = path + 'train.csv'
unlabeled_name = path + 'unlabeled.csv'
dev_name = path + 'dev.csv'
##test_name = path + 'test.csv'

train.to_csv(train_name, index=False, header=False)
unlabeled.to_csv(unlabeled_name, index=False, header=False)
dev_df.to_csv(dev_name, index=False, header=False)
# test_data.to_csv(test_name, index=False, header=False)